In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Ayush\\OneDrive\\Desktop\\Coccidiosis-chicken-disease-classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Ayush\\OneDrive\\Desktop\\Coccidiosis-chicken-disease-classification'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path 
    all_params: dict
    params_image_size: int
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [10]:
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(rescale=1./255,
                                    validation_split=0.30)
        
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1],
                               batch_size=self.config.params_batch_size,
                               interpolation="bilinear")
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        model = tf.keras.models.load_model(path)
        return model
    
    def evaluate(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {
            "loss": float(self.score[0]),
            "accuracy": float(self.score[1])
        }
        save_json(path=Path("scores.json"), data=scores)


In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()

    evaluation = Evaluation(val_config)
    evaluation.evaluate()
    evaluation.save_score()
except Exception as e:
    raise e


# to remove the error, we need to comment out the @ensure_annotation in common.py save_json function

Found 116 images belonging to 2 classes.
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - accuracy: 0.9654 - loss: 0.1121
